In [1]:
from obspy.clients.fdsn import Client
client = Client("http://eida.ethz.ch")

In [223]:
opt={'starttime':'2020-01-01T00:00:00',
     'minlatitude':40.8004,
     'maxlatitude':47.8067,
     'minlongitude':5.9362,
     'maxlongitude':10.4958,
     'minmagnitude':1.5,
     'orderby':'magnitude',
     'eventtype':'earthquake'}
catalog=client.get_events(**opt)
for i,e in enumerate(catalog):
      fopt={'includeallorigins':True,
            'includeallmagnitudes':True}
      catalog[i]=client.get_events(eventid=e.resource_id.resource_id,**fopt)[0]

In [233]:
catalog

460 Event(s) in Catalog:
2020-10-25T19:35:43.383892Z | +46.905,   +9.125 | 4.324859837 MLh | manual
2021-10-05T05:39:25.176962Z | +45.975,   +7.515 | 4.114662185 MLhc | manual
...
2021-06-03T16:19:38.563176Z | +46.227,   +7.532 | 1.503589118 MLhc | manual
2021-07-01T11:45:05.551089Z | +46.622,   +8.372 | 1.503187679 MLhc | manual
To see all events call 'print(CatalogObject.__str__(print_all=True))'

In [22]:
%matplotlib widget

In [250]:
import matplotlib.pyplot,numpy

for e in catalog: 
    e.myevent=False
    e.myorigins=False
    for o in e.origins:
        o.mymagnitudes=[]
        o.mymagnitudetypes=[]
        o.mymagnitudedelays=[]
        o.myauthor='None'
        o.myorigin=False
        o.mydelay=o.creation_info.creation_time - e.preferred_origin().time
        o.mytime=e.preferred_origin().time
        o.mymag=e.preferred_magnitude().mag

    if not isincountries(e.preferred_origin().latitude,
                            e.preferred_origin().longitude,'ch,li'):
        print('Not in Ch,Li:',e.short_str())
        for o in e.origins:
            o.myauthor='Out'
            o.mymagnitudetypes = 'None'
        continue

    e.myevent=True
    for o in e.origins:
        author=o.creation_info.author
        if author is None:
            continue
        if 'NLo' not in author and 'NTeT' not in author:
            continue
        if 'auloc' not in author:
            continue
        e.myorigins=True
        o.myorigin=True
        o.myauthor=o.creation_info.author.split('@')[0] 

    if not e.myorigins:
        print('No N(Lo,Te)(N,B,T):',e.short_str())
    
    for m in e.magnitudes:
        o = m.origin_id.get_referred_object()
        author=o.creation_info.author
        if not o.myorigin:
            continue
        if 'MLh' not in m.magnitude_type:
            continue
        if m.station_count<1:
            continue
        o.mymagnitudes+=[m]
        o.mymagnitudetypes+=[m.magnitude_type[1:]]
        o.mymagnitudedelays+=[m.creation_info.creation_time -  e.preferred_origin().time]
        

    for o in e.origins:
        if not o.myorigin:
            o.mymagnitudetypes = 'None'
            continue
        if not len(o.mymagnitudes):
            o.mymagnitudetypes = 'None'
            #print('No M:',e.short_str())
            continue
        if 'Lhc' not in o.mymagnitudetypes:
            pass#rint('No MLhc:',e.short_str())
        if 'Lh' not in o.mymagnitudetypes:
            pass#rint('No MLh:',e.short_str())
        o.mymagnitudetypes = ','.join(list(set(o.mymagnitudetypes)))

authors = [o.myauthor for e in catalog for o in e.origins if e.myevent if o.myorigin ]
magnitude_types = [o.mymagnitudetypes for e in catalog for o in e.origins if e.myevent if o.myorigin ]
uauthors=list(set(authors))[-1::-1]
umtypes=list(set(magnitude_types))[-1::-1]
print(uauthors)
print(umtypes)

sizes=[o.mymag for e in catalog for o in e.origins if e.myevent ]
def m2s(m):
    return (m-min(sizes))/(max(sizes)-min(sizes))*100+10 
times=list(numpy.sort([e.origins[0].mytime for e in catalog if e.myevent ]))
def t2y(t):
    return times.index(t)
delays=[o.mydelay for e in catalog for o in e.origins if e.myevent if o.myorigin if o.mydelay>0]

fig, ax = matplotlib.pyplot.subplots(nrows=1, ncols=len(uauthors), 
                                     sharex=True, sharey=True, 
                                     gridspec_kw={'wspace':0},
                                     constrained_layout=True,
                                     figsize=(20,len(times)*.2))

for i,a in enumerate(uauthors): 
    ax[i].set_xlim([min(delays),max(delays)])
    ax[i].set_xscale('log')
    ax[i].tick_params(right=True, top=True,left=True, bottom=True,labelbottom=True, labeltop=True,labelright=True, labelleft=True,which='both')
    ax[i].xaxis.label.set_size('small')
    ax[i].set_yticks(range(len(times)))
    ax[i].set_yticklabels([str(t).split('.')[0].replace('T','\n$^{')+'}$' for t in times],fontsize='x-small')
    
    if i==0:
        ax[i].tick_params(labelright=False)
    elif i==len(uauthors)-1:
        ax[i].tick_params(labelleft=False)
    else:
        ax[i].tick_params(labelright=False,labelleft=False)
    ax[i].grid(b=True, which='major', color='gray', linestyle='dashdot', zorder=-999)
    ax[i].grid(axis='x', which='minor', color='beige',  ls='-', zorder=-999)
    #ax[i].yaxis.set_minor_locator(matplotlib.ticker.AutoMinorLocator())

    for t in umtypes: 
        x=[o.mydelay for e in catalog for o in e.origins if e.myevent if o.myorigin if o.myauthor == a if o.mymagnitudetypes == t]
        y=[t2y(o.mytime) for e in catalog for o in e.origins  if e.myevent if o.myorigin if o.myauthor == a if o.mymagnitudetypes == t]
        s=[m2s(o.mymag) for e in catalog for o in e.origins if e.myevent if o.myorigin if o.myauthor == a if o.mymagnitudetypes == t]
        if not len(x):
            continue
        ax[i].scatter(x,
                y,
                s=s, 
                color=['g','orange','r'][umtypes.index(t)],#'C%s'%str(umtypes.index(t)+1),
                marker='|',#str(uauthors.index(a)+1),
                label="%s\nM[%s]"%(a,t),
                capstyle='round',
                zorder=9-len(t))
    for m in range(20): 
        if m/2.<min(sizes):
            continue
        if m/2.>max(sizes):
            continue
        ax[i].scatter(numpy.nan,
                numpy.nan,
                    s=m2s(m/2), 
                    capstyle='round',
                    color='k',
                    marker='|',
                    label="M%s"%(m/2))
    ax[i].legend(fontsize='small')
fig.supxlabel('Time after origin time (second)',fontsize='small')
fig.suptitle('Earthquakes in Ch,Li with M≧%.1f'%min(sizes))
ax[0].set_ylim([-1,len(times)])

Not in Ch,Li: 2020-06-23T06:25:41.127671Z | +46.039,   +6.919 | 3.8248257 MLh | manual
Not in Ch,Li: 2021-02-05T14:14:11.661805Z | +47.751,   +8.825 | 3.22006313 MLhc | manual
Not in Ch,Li: 2021-03-01T19:43:36.475270Z | +47.686,   +9.061 | 3.076720445 MLhc | manual
Not in Ch,Li: 2020-05-18T11:14:30.915812Z | +46.205,   +6.410 | 2.934959032 MLh | manual
Not in Ch,Li: 2020-12-01T18:05:35.785473Z | +47.713,   +7.774 | 2.84819265 MLhc | manual
Not in Ch,Li: 2020-10-02T09:52:25.008181Z | +45.743,   +7.010 | 2.822048467 MLh | manual
Not in Ch,Li: 2020-06-22T05:47:00.688793Z | +47.018,   +6.234 | 2.810470805 MLh | manual
Not in Ch,Li: 2020-11-14T07:27:40.614516Z | +46.056,   +6.596 | 2.759636495 MLhc | manual
Not in Ch,Li: 2020-10-26T20:47:56.743634Z | +45.568,   +6.847 | 2.731130941 MLh | manual
Not in Ch,Li: 2020-12-25T17:15:04.656688Z | +45.704,   +6.929 | 2.727602167 MLhc | manual
Not in Ch,Li: 2021-05-02T12:24:29.082345Z | +45.826,   +7.449 | 2.654238753 MLhc | manual
Not in Ch,Li: 2020-

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

(-1.0, 332.0)

In [251]:
fig.savefig('plots/magnitudes.jpg',
             bbox_inches='tight')#dpi=512,